In [1]:
# 패키지 설치
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# 데이터 불러오기
log_data = pd.read_csv('/content/gdrive/MyDrive/BigContest_data/log_data.csv')
log_data

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,iOS,3.6.1,2022-03-25
...,...,...,...,...,...,...
17843988,242442,StartLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
17843989,242442,EndLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
17843990,593062,OpenApp,2022-05-27 16:19:10,Android,3.13.0,2022-05-27
17843991,593062,Login,2022-05-27 16:19:15,Android,3.13.0,2022-05-27


In [4]:
log_data.event.unique()

array(['StartLoanApply', 'ViewLoanApplyIntro', 'EndLoanApply', 'OpenApp',
       'UseLoanManage', 'UsePrepayCalc', 'Login',
       'CompleteIDCertification', 'UseDSRCalc', 'SignUp', 'GetCreditInfo'],
      dtype=object)

In [5]:
grouped = log_data.groupby(['user_id', 'event'])['timestamp'].min()
grouped

user_id  event             
1        GetCreditInfo         2022-05-03 14:52:28
         Login                 2022-06-16 23:58:41
         UseLoanManage         2022-06-16 23:58:41
7        GetCreditInfo         2022-05-22 16:39:49
9        GetCreditInfo         2022-05-21 23:37:58
                                      ...         
879696   UseLoanManage         2022-03-14 05:35:41
         ViewLoanApplyIntro    2022-03-14 05:37:55
879698   OpenApp               2022-05-24 22:33:24
         StartLoanApply        2022-05-24 22:33:32
         ViewLoanApplyIntro    2022-05-24 22:33:32
Name: timestamp, Length: 2860127, dtype: object

In [6]:
pd.DataFrame(grouped)

timestamp
user_id event                                  
1       GetCreditInfo       2022-05-03 14:52:28
        Login               2022-06-16 23:58:41
        UseLoanManage       2022-06-16 23:58:41
7       GetCreditInfo       2022-05-22 16:39:49
9       GetCreditInfo       2022-05-21 23:37:58
...                                         ...
879696  UseLoanManage       2022-03-14 05:35:41
        ViewLoanApplyIntro  2022-03-14 05:37:55
879698  OpenApp             2022-05-24 22:33:24
        StartLoanApply      2022-05-24 22:33:32
        ViewLoanApplyIntro  2022-05-24 22:33:32

[2860127 rows x 1 columns]

In [8]:
funnel_steps = pd.DataFrame(data = {'step_no':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]},index = ['SignUp','OpenApp','Login','ViewLoanApplyIntro', 'StartLoanApply', 'CompleteIDCertification', 'EndLoanApply', 'UseLoanManage','UsePrepayCalc', 'UseDSRCalc','GetCreditInfo'])
funnel_steps

,step_no
SignUp,1
OpenApp,2
Login,3
ViewLoanApplyIntro,4
StartLoanApply,5
CompleteIDCertification,6
EndLoanApply,7
UseLoanManage,8
UsePrepayCalc,9
UseDSRCalc,10


In [9]:
grouped = pd.DataFrame(grouped).merge(funnel_steps, left_on='event', right_index = True)
grouped

,,timestamp,step_no
user_id,event,,
1,GetCreditInfo,2022-05-03 14:52:28,11
7,GetCreditInfo,2022-05-22 16:39:49,11
9,GetCreditInfo,2022-05-21 23:37:58,11
11,GetCreditInfo,2022-03-24 10:54:07,11
12,GetCreditInfo,2022-05-01 18:30:42,11
...,...,...,...
878796,UseDSRCalc,2022-04-05 21:48:33,10
878870,UseDSRCalc,2022-04-22 18:54:29,10
879287,UseDSRCalc,2022-04-05 17:30:02,10


In [10]:
funnel = grouped.reset_index().pivot(index = 'user_id', columns = 'step_no', values = 'timestamp')
funnel

step_no,1,2,3,4,5,6,7,8,9,10,11
user_id,,,,,,,,,,,
1,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,2022-05-03 14:52:28
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-22 16:39:49
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-21 23:37:58
11,NaN,2022-03-24 10:53:59,NaN,2022-03-24 10:55:43,2022-03-24 10:55:43,2022-03-24 10:56:26,2022-03-24 10:59:46,2022-03-24 10:54:08,2022-03-24 10:54:36,NaN,2022-03-24 10:54:07
12,NaN,2022-03-14 01:13:11,2022-03-14 01:13:15,2022-03-14 01:13:22,2022-03-14 01:13:21,2022-03-14 01:13:46,2022-03-14 01:16:26,2022-05-01 18:30:48,NaN,NaN,2022-05-01 18:30:42
...,...,...,...,...,...,...,...,...,...,...,...
879693,NaN,2022-05-13 11:40:29,NaN,2022-05-13 11:29:49,2022-05-13 11:29:49,2022-05-13 11:30:06,2022-05-13 11:34:29,2022-05-13 13:24:32,NaN,NaN,2022-05-13 13:24:33
879694,NaN,NaN,NaN,2022-03-31 20:07:23,2022-03-31 20:07:23,2022-03-31 20:07:42,2022-03-31 20:18:18,NaN,NaN,NaN,NaN
879695,NaN,NaN,NaN,2022-05-27 12:48:32,2022-05-27 12:48:32,2022-05-27 12:48:51,2022-05-27 12:51:23,NaN,NaN,NaN,NaN


In [11]:
funnel.columns  = funnel_steps.index
funnel

,SignUp,OpenApp,Login,ViewLoanApplyIntro,StartLoanApply,CompleteIDCertification,EndLoanApply,UseLoanManage,UsePrepayCalc,UseDSRCalc,GetCreditInfo
user_id,,,,,,,,,,,
1,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,2022-05-03 14:52:28
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-22 16:39:49
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-21 23:37:58
11,NaN,2022-03-24 10:53:59,NaN,2022-03-24 10:55:43,2022-03-24 10:55:43,2022-03-24 10:56:26,2022-03-24 10:59:46,2022-03-24 10:54:08,2022-03-24 10:54:36,NaN,2022-03-24 10:54:07
12,NaN,2022-03-14 01:13:11,2022-03-14 01:13:15,2022-03-14 01:13:22,2022-03-14 01:13:21,2022-03-14 01:13:46,2022-03-14 01:16:26,2022-05-01 18:30:48,NaN,NaN,2022-05-01 18:30:42
...,...,...,...,...,...,...,...,...,...,...,...
879693,NaN,2022-05-13 11:40:29,NaN,2022-05-13 11:29:49,2022-05-13 11:29:49,2022-05-13 11:30:06,2022-05-13 11:34:29,2022-05-13 13:24:32,NaN,NaN,2022-05-13 13:24:33
879694,NaN,NaN,NaN,2022-03-31 20:07:23,2022-03-31 20:07:23,2022-03-31 20:07:42,2022-03-31 20:18:18,NaN,NaN,NaN,NaN
879695,NaN,NaN,NaN,2022-05-27 12:48:32,2022-05-27 12:48:32,2022-05-27 12:48:51,2022-05-27 12:51:23,NaN,NaN,NaN,NaN


In [12]:
step_values = [funnel[column].notnull().sum() for column in funnel.columns]
step_values

[28232,
 418602,
 347643,
 356475,
 326555,
 311972,
 302466,
 297468,
 4486,
 3619,
 462609]

* 지금 마지막에 나온게 각 단계 별 user_id 개수 인데, 아무래도 막 쇼핑처럼 로그인-> 고르기-> 장바구니 -> 구매 이런 명확한 과정이 없다보니까 고객 이탈률을 보기는 좀 어려워 보이고
* 그나마 얻을 수 있는 정보는, 1. 대출 관련 서비스 이용보다는 GetCreditInfo(신용조회)만 하는 사람이 특히 많다, 2. UsePrepayCalc, 	UseDSRCalc 이런건 사람들이 별로 사용을 안한다